In [1]:
import gzip
import simplejson
import pandas as pd
import numpy as np
import re
import RAKE
import nltk

from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from summa import keywords
from gensim.summarization import keywords as kw
from nltk.tokenize import RegexpTokenizer
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
import string

nltk.download('stopwords')
stop = stopwords.words('english')
rake = RAKE.Rake(stop)
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = MorphAnalyzer()

def normalize_text(text):
    text = text.replace("\n", " ").replace('/', ' ')
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    lemmas = [lemmatizer.parse(t)[0].normal_form for t in text.split()]
    lemmas = [i for i in lemmas if not i.isdigit()]
    return ' '.join(lemmas)


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1051)>


In [2]:
buf = []

with open("Office_Products.txt", "r") as f:
    entry = {}
    for line in f.readlines():
        line = line.strip()
        colonPos = line.find(':')
        if colonPos == -1:
            buf.append(entry)
            entry = {}
            continue
        eName = line[:colonPos]
        rest = line[colonPos+2:]
        entry[eName] = rest
    buf.append(entry)
    
data = pd.DataFrame.from_dict(buf)


In [3]:
data = data[:20000]

In [4]:
data.head()

,product/price,product/productId,product/title,review/helpfulness,review/profileName,review/score,review/summary,review/text,review/time,review/userId
0,unknown,B000E7F8LA,"Low Odor Dry Erase Markers, Vibrant DryGuard I...",0/0,S. Young,4.0,Boone Dry Erase Markers,I was really happy to find out that I was stil...,1214784000,A266N1TVOHUG8V
1,17.94,B000CD483K,C-Line Clear 62033 Heavyweight Antimicrobial P...,0/0,"Thomas Perrin ""Perrin & Treggett""",5.0,Superior product for archival storage,Ever since some of my important family documen...,1353628800,A1186EZQ23CU4X
2,443.04,B0006Q9950,Wasp Barcode Technologies 633808920128 Cordles...,14/14,Handyman,4.0,"Good product, bad instructions",My boss had us using cheap USB barcode scanner...,1320364800,A2CW9GKMNFAU6R
3,13.99,B0001YXWV4,Panasonic MARKER ERASER KIT 1 EA-BLK RED BLU E...,0/0,C L Huddleston,5.0,Best markers made,We use our white boards every day. Tired of ma...,1359676800,A14XEQHPPULFDA
4,13.99,B0001YXWV4,Panasonic MARKER ERASER KIT 1 EA-BLK RED BLU E...,0/0,Eiji Nakamura,5.0,Good item,Fast shipment and fast response.The item is go...,1358294400,A7YN96KKCI8GO


In [5]:
data['normal_title'] = data['product/title'].apply(normalize_text)
data['normal_text'] = data['review/text'].apply(normalize_text)

### (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара. Могут помочь заголовки и дополнительные данные с Amazon (Metadata здесь) Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

1) (будет реализован) Взять слова из title каждого товара и найти для них все пары употребления (ообычно в заглавии содержатся основные слова: название модели, фирма, общее назначение, категория и т. п.) и собрать употребляющиеся с ним рядом слова из отзыва (перед этим нормализовав все тексты). Достоинствами метода является хорошая интерпретируемость и регулирование выделяемых n-грамм. В качестве минусов можно выделить отсутствие регулируемых параметров (но можно использовать параллельно с другими способами, которые даже в совокупности можно легко реализовать).

2) Подход, основанных на правилах, в котором необходимо построить правила и с помощью парсера (который выделял бы нужные элементы из текста) извлечь соответствующие элементы. Могло бы помочь описание/свойства устройства. Достоинствами метода являются регулируемые правила, которые позволяют доставать конкретные параметры устройства и лёгкая формализация. Минусами данного подхода являются отсутствие знаний о метаданных и структурах отзывов, что повлечёт за собой изучение исходных примеров для выявления работающих правил.

3) Использование готовых решений из библиотек, основанных на нейросетях. Достоинствами такого подхода являются нахождение сложных зависимостей и большое количество регулируемых параметров (например, для каждой конкретной категории). Минусами являются скорость работы и невозможность интерпретировать подход к нахождению сущностей, также зачастую будут выделятся только named entity, а общие слова (принтер, сканер и т п) будут пропускаться.


### (2 балла) Реализуйте один из предложенных вами способов.

### (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

In [6]:
collocations = []
id_keywords = {}

for name in data['product/productId'].unique():
    keywords = data['normal_title'][data['product/productId'] == name].values[0].split()
    review_words = data['normal_text'][data['product/productId'] == name].values[0].split()
    
#     print(review_words)
    for ind, word in enumerate(review_words):
        if word in keywords:
            if 1 < ind:
                collocations.append(review_words[ind - 1] + " " + word)
            if ind < len(review_words) - 2:
                collocations.append(word + " " + review_words[ind + 1])
                

Оставим все уникальные коллокации

In [7]:
unique_collocations = set(collocations)
len(unique_collocations)

9103

### (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика ранжирует выше коллокации, подходящие для отчёта).

Инициализируем необходимые классы и функцию, которая возвращает только те биграммы, которые мы выделили из отзывов

In [8]:
bigram_measures = BigramAssocMeasures()
bigram_finder = BigramCollocationFinder.from_documents([i.split() for i in data['normal_text'].values])
bigram_finder.apply_freq_filter(10)

In [9]:
def get_our_bigram(my_set, finder):
    res = []
    for value in finder:
        if value[0][0] + ' ' + value[0][1] in my_set:
            res.append(value)
    return res

Показать view_top_n экземпляров с наивысшим скоором

In [10]:
view_top_n = 15

#### likelihood ratio

In [11]:
bigram_finder_like = bigram_finder.score_ngrams(bigram_measures.likelihood_ratio)
get_our_bigram(unique_collocations, bigram_finder_like)[:view_top_n]

[(('if', 'you'), 20165.278169880785),
 (('i', 'have'), 18010.605080651658),
 (('this', 'phone'), 14421.019309193884),
 (('caller', 'id'), 14246.752822223207),
 (('easy', 'to'), 13129.146250369971),
 (('on', 'the'), 12016.587579175814),
 (('of', 'the'), 11534.601231167548),
 (('it', 'is'), 10788.412438246625),
 (('i', 'bought'), 10713.261325363601),
 (('a', 'lot'), 9871.753396066808),
 (('answering', 'machine'), 9678.644699830651),
 (('to', 'use'), 9548.964043632008),
 (('to', 'be'), 8380.503721501605),
 (('a', 'few'), 7670.070973270589),
 (('in', 'the'), 7581.8427298064125)]

#### PMI

In [12]:
bigram_finder_pmi = bigram_finder.score_ngrams(bigram_measures.pmi)
get_our_bigram(unique_collocations, bigram_finder_pmi)[:view_top_n]

[(('van', 'gogh'), 15.96602304429172),
 (('vision', 'elite'), 15.398982451567827),
 (('hello', 'kitty'), 14.75852483825497),
 (('obus', 'forme'), 14.257626602322286),
 (('stainless', 'steel'), 13.907841973203196),
 (('movie', 'writer'), 13.32150369889011),
 (('dr', 'grip'), 12.805031167619418),
 (('fellowes', 'powershred'), 12.695375454348055),
 (('bubble', 'wrap'), 12.65519265828769),
 (('double', 'sided'), 12.6000247036674),
 (('step', 'stool'), 12.591627529510221),
 (('hearing', 'aid'), 12.166321694777553),
 (('poly', 'mailers'), 12.10620070233998),
 (('coin', 'sorter'), 12.099737793771757),
 (('filing', 'cabinet'), 12.044746035977905)]

#### dice

In [13]:
bigram_finder_dice = bigram_finder.score_ngrams(bigram_measures.dice)
get_our_bigram(unique_collocations, bigram_finder_dice)[:view_top_n]

[(('van', 'gogh'), 0.851063829787234),
 (('caller', 'id'), 0.70893760539629),
 (('obus', 'forme'), 0.6181818181818182),
 (('stainless', 'steel'), 0.6122448979591837),
 (('answering', 'machine'), 0.5532533624136677),
 (('vision', 'elite'), 0.4878048780487805),
 (('laser', 'pointer'), 0.47173489278752434),
 (('hello', 'kitty'), 0.4657534246575342),
 (('customer', 'service'), 0.45029624753127057),
 (('movie', 'writer'), 0.4142857142857143),
 (('heavy', 'duty'), 0.40877598152424943),
 (('dry', 'erase'), 0.3744493392070485),
 (('mouse', 'pad'), 0.3397152675503191),
 (('bubble', 'wrap'), 0.32044198895027626),
 (('step', 'stool'), 0.32038834951456313)]

#### Хи-квадрат

In [14]:
bigram_finder_chi_sq = bigram_finder.score_ngrams(bigram_measures.chi_sq)
get_our_bigram(unique_collocations, bigram_finder_dice)[:view_top_n]

[(('van', 'gogh'), 0.851063829787234),
 (('caller', 'id'), 0.70893760539629),
 (('obus', 'forme'), 0.6181818181818182),
 (('stainless', 'steel'), 0.6122448979591837),
 (('answering', 'machine'), 0.5532533624136677),
 (('vision', 'elite'), 0.4878048780487805),
 (('laser', 'pointer'), 0.47173489278752434),
 (('hello', 'kitty'), 0.4657534246575342),
 (('customer', 'service'), 0.45029624753127057),
 (('movie', 'writer'), 0.4142857142857143),
 (('heavy', 'duty'), 0.40877598152424943),
 (('dry', 'erase'), 0.3744493392070485),
 (('mouse', 'pad'), 0.3397152675503191),
 (('bubble', 'wrap'), 0.32044198895027626),
 (('step', 'stool'), 0.32038834951456313)]

Наиблее качественными выглядят результаты работы Dice, выделяется больше качественных сочетаний и показывается информация о конкретных моделях и фирмах, likelihood выделяет по частотности слов и, как следствие, биграммы со стоп-словами попадают в топ.

### (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров. Должны получиться примерно такие группы:

In [15]:
def get_top_by_word(my_set, finder, word):
    res = []
    for value in finder:
        if value[0][0] == word or value[0][1] == word:
            res.append(value[0][0] + " " + value[0][1])
    return res

In [16]:
words = ['sharpener', 'card', 'printer', 'phone', 'pencil']
get_top_n = 5

for word in words:
    print("\nWord: ", word)
    print(*get_top_by_word(unique_collocations, bigram_finder_dice, word)[:get_top_n], sep='\n')
    print()


Word:  sharpener
pencil sharpener
electric sharpener
xacto sharpener
this sharpener
sharpener has


Word:  card
credit card
card stock
business card
bookman card
card holder


Word:  printer
laser printer
inkjet printer
hp printer
canon printer
photo printer


Word:  phone
this phone
cordless phone
the phone
speaker phone
phone system


Word:  pencil
pencil sharpener
electric pencil
pencil sharpeners
mechanical pencil
pencil case



### Бонус (2 балла): если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

Синонимичные упоминания можно детектировать с помощью расстояния между векторами, если получить эмбеддинги соответствующих слов (word2vec, fasttext и т п), но проблема в редких употреблениях некоторых специфичных слов в предобученных моделях (smartwatch и watch) - но это проблему можно решить дообучением на интересующих текстах. Также употребление прилагательных, глаголов можно определить с помощью методов выделения частей речи, что позволит фильтровать такие сочетания (NOUN + VERB/ADJ) и брать из них только существительное.